# IN4080: obligatory assignment 1 (Autumn 2025)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Friday, September 12 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

First, make sure that all required modules are installed:

In [1]:
%pip install tqdm numpy scikit_learn

Note: you may need to restart the kernel to use updated packages.


## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [2]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

    # Implement here your basic tokenisation
    text = re.sub(r'([\.\,\:\;\!\?\(\)\-\"])', r' \1 ', text) #here we add a white space around the punctuation markers and other symbols, so they will be treated as tokens
    tokens = text.split()
    return tokens

In [3]:
line =  """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

print(basic_tokenize(line)) #simply run the function from above to split the sentence into tokens

['The', 'method', 'should', 'split', 'the', 'text', 'on', 'white', 'space', ',', 'except', 'for', 'punctuation', 'markers', 'that', 'should', 'be', 'considered', 'as', 'tokens', 'of', 'their', 'own', '(', 'even', 'in', 'the', 'absence', 'of', 'white', 'space', 'before', 'or', 'after', 'their', 'occurrence', ')']


We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 
There were 259220 tokens extracted from the text file, and 30054 distinct types.

In [4]:
# read the text file
with open('ndt_test_lm.txt') as f:
    ndt_test_lm_txt = f.read()

# run the funtion from above
ndt_test_lm_tokens = basic_tokenize(ndt_test_lm_txt)

# count the number of tokens extracted: 259220
print(len(ndt_test_lm_tokens))

#count how many distinct words are there: 30054
print(len(set(ndt_test_lm_tokens)))

259220
30054


We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [5]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

Using the basic tokenizer we extracted 259220 tokens with 30054 types. While using the BPE tokenizer the number of the extracted tokens is 386099 with 4408 types. The number of tokens has increased a lot, and this is because we split words into smaller bytes using the BPE tokenizer. However, the number of distinct words has decreased because many words share the same bytes that have been tokenized. 

In [6]:
# training the BPE tokenizer using the training dataset 
bpe_train = BPETokenizer("ndt_train_lm.txt",5000)

# read the file and test the tokenizer on the test dataset 
with open('ndt_test_lm.txt', 'r') as f:
    ndt_test_lm_txt = f.read()

# here we tokenize the test text using the trained tokenizer
bpe_test = BPETokenizer('ndt_test_lm.txt')
tokens_test = list(bpe_train(ndt_test_lm_txt))

#count the number of tokens and types
print(len(tokens_test)) # 386099
print(len(set(tokens_test))) #4408


Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

Counting word occurrences in corpus ndt_test_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226096 [00:00<?, ?it/s]

386099
4408


__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

In [7]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

########### modifications #############
# define the punctuations 
import string
print(string.punctuation)
#######################################

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])

########################################################### modifications #####################################################################
 # here we detect if there are punctuation markers 
    def is_punctuation(self, character: str) -> bool:
            """Check if a character is punctuation"""
            return character in string.punctuation

    # in this function we decide whether 2 subwords can be merged together without contacting the words and punctuations
    def can_merge(self, left: str, right: str) -> bool:
            """Check if two subwords can be merged based on punctuation rules"""
            # define the last character of left subword and first character of right subword which can potentially be contacted to each other
            left_last = left[-1] if left else ''
            right_first = right[0] if right else ''
            
            # check if the combination consists of a letter and a punctuation marker regardless of the direction
            if (self.is_punctuation(left_last) and right_first.isalpha()) or \
            (left_last.isalpha() and self.is_punctuation(right_first)):
                return False
            
            return True

###################################################################################################################################################


    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])

 ##################################################################################### modifications ##############################################################################################
                if self.can_merge(subwords[i], subwords[i+1]): # this adds an condition that only if the combinations are not containing the punctuation markers can continue in later operations
 ##################################################################################################################################################################################################
                                     
                    pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

############################################################################ modifications ################################################################
            valid_pairs = [pair for pair in pairs if self.can_merge(pair[0], pair[1])]  #define and use the valid pairs in the following steps of splitting
            if not valid_pairs:
                break

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(valid_pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
############################################################################################################################################################

            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published last year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

1. why can't LLM spell words & why it can't reverse a string
The LLMs can not spell words because tokenization splits words into tokens instead of single letters. The LLM predicts the tokens but it does not know the structure of the words letter by letter. 
2. Why is LLM worse at non-English languages
We might create more tokens for the non-English languages than the English, because the non-englishes do not have spaces between words and each character might has a meaning, which makes it harder to combine the subwords. Consequently the model is less efficient than the well trained english wokenizations.
3. why is LLM bad at simple arithmetic? 
This is because the numbers can be tokenized differently than what they actually are. For example 123 might be tokenized to 1 and 23, which does not help calculating the true number. 
4. What is this weird warning I get about a "trailing whitespace"?
Tokenization handles the word with a trailing whitespace differently than the word itself. For example, "tokennization" and "tokenization " could be different tokens which impaires the predictions of the models.

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [8]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()
 

    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [9]:
# In this task i need to (1) tokenize the corpus, (2) add <s> in the begining for the first token, (3)count the n grams and n-1 grams, and (4) calculate the token probabilities with smoothing 

from collections import defaultdict, Counter

def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=2, alpha_smoothing=0.1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        self.alpha_smoothing = alpha_smoothing
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:  
            
            # ADD HERE YOUR CODE TO FILL THE VALUES IN self.ngram_probs
            # first in this step we count the number of n grams and the contexts 
            c_ngram = defaultdict(int)
            c_context = defaultdict(int)

            for line in fd:
                line = line.strip()
                if not line:
                    continue

                # Tokenization
                tokens = list(tokenizer(line))

                # adding <s> tokens in the beginning for context so that the first token can be recognized 
                if self.ngram_size > 1:
                    tokens = ["<s>"] * (self.ngram_size - 1) + tokens

                # to estimate the probabilities of the next token, we use the formula: p(next token|previous tokens) = (count(the previous tokens followed by the next token))/(count(the previous tokens))
                # in this step we count the numerator and denominator in the formula
                for i in range(len(tokens) - self.ngram_size + 1): # here i is the position number in the n gram 
                    ngram = tuple(tokens[i:i + self.ngram_size]) # define the ngram tuple
                    context = ngram[:-1] # here we define the previous tokens in the formula n-1
                    c_ngram[ngram] += 1 # here we define the number of ngrams counts
                    c_context[context] += 1 # we count how many times the previous tokens have occurred 

            # now calculating the probabilities using laplace smoothing
            V = len(tokenizer.vocab) # this is the number of unique tokens
            alpha = self.alpha_smoothing # this defines the degree of alpha 
            
            for context, ctx_count in c_context.items():
                self.ngram_probs[context] = {}
                for token in tokenizer.vocab:
                    count = c_ngram[(context + (token,))]
                    self.ngram_probs[context][token] = (count + alpha) / (ctx_count + alpha * V)    

setattr(NGramLanguageModel, '__init__', __init__)



__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [10]:
# define the bpe_tokenizer
bpe_train = BPETokenizer("ndt_train_lm.txt",5000)
# training the n gram model
language_model = NGramLanguageModel(
    training_corpus_file='ndt_train_lm.txt',
    tokenizer=bpe_train,
    ngram_size=3,           
    alpha_smoothing= 1     # Laplace smoothing alpha =1
)

# read the dataset 
with open('ndt_test_lm.txt') as f:
    ndt_test_lm = f.read()

# compute the perplexity
perplexity = language_model.get_perplexity(ndt_test_lm)
print(perplexity)

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/872824 [00:00<?, ?it/s]

__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [ ]:
# first we specify two lists for later use
sentences = []
labels = []

#import the training dataset 
with open('ndt_train_class.txt') as f:
    for line in f:
        line = line.strip()
        if '\t' in line:
            sentence, label = line.rsplit('\t', 1) # here we split the line into the norwegian sentence and the label of the type of norwegian
            sentences.append(sentence) # the norwegian sentence
            labels.append(1 if label == 'nno' else 0)  # here we convert the labels: Nynorsk = 1, Bokmaal = 0

# then we convert the labels to np array as the training outcome
y_train = np.array(labels)

# Create vocabulary mapping using the bpe tokenizer from before
vocab_to_idx = {token: idx for idx, token in enumerate(bpe_train.vocab)}

# creating the NxV matrix 
N = len(sentences)  # the number of N instances
V = len(bpe_train.vocab)  # the size of the vocabulary 
X_train = np.zeros((N, V)) # create the array with 0s

# then we fill the matrix with binary numbers
for i, sentence in enumerate(sentences):
    # Tokenize the sentence
    tokens = list(bpe_train(sentence, show_progress_bar=False))
    
    # Mark presence of each token with 0 absent
    for token in tokens:
        if token in vocab_to_idx:
            X_train[i, vocab_to_idx[token]] = 1



: 

__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [ ]:
from sklearn.linear_model import LogisticRegression

#train the logistic regression model using the training dataset 
classifier = LogisticRegression(solver='liblinear', random_state=1)
classifier.fit(X_train, y_train)



: 

__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
# define 2 lists for later use 
test_sentences = []
test_labels = []

# read the test set and do the same as we did on training data
with open('ndt_test_class.txt') as f:
    for line in f:
        line = line.strip()
        if '\t' in line:
            sentence, label = line.rsplit('\t', 1)
            test_sentences.append(sentence)
            test_labels.append(1 if label == 'nno' else 0)

# similarly create the feature matrix for test data
N_test = len(test_sentences)
X_test = np.zeros((N_test, V))

for i, sentence in enumerate(test_sentences):
    tokens = list(bpe_train(sentence, show_progress_bar=False))
    for token in tokens:
        if token in vocab_to_idx:
            X_test[i, vocab_to_idx[token]] = 1

# now we predict the outcome usinhg the trained logistic regression model 
y_pred = classifier.predict(X_test)

# and here we measure the goodness of the model prediction  
accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)

print(accuracy)
print(precision)
print(recall)

: 

__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [ ]:
# inspect the weights by coef
coefficients = classifier.coef_[0]
print(coefficients)

# creating reverse mapping from index back to token
idx_to_vocab = {idx: token for token, idx in vocab_to_idx.items()}

# sorting the index in terms of the coefficients
sorted_indices = np.argsort(coefficients)

# get the top 5 features for nynorsk with the highest weights
nynorsk_indices = sorted_indices[-5:][::-1]  
for idx in nynorsk_indices:
    token = idx_to_vocab[idx]
    weight = coefficients[idx]
    print(f"  '{token}': {weight:.4f}")

# get the top 5 features for bokmaal with the negative values
bokmal_indices = sorted_indices[:5] 
print("\nTop 5 subwords contributing to Bokmål classification:")
for idx in bokmal_indices:
    token = idx_to_vocab[idx]
    weight = coefficients[idx]
    print(f"  '{token}': {weight:.4f}")

# yes these weights make sense, because they look like the most frequntly used words and also differentiate the nynorsk and bokmaal. But i am not sure why "eg" is not shown in the nynorsk list and "jeg" & "Jeg" in bokmaal are repetitive.

[ 0.          0.          0.         ... -0.32454984  0.42937562
  0.30562655]
  'ikkje ': 4.2111
  'frå ': 4.0288
  'eit ': 3.8644
  'dei ': 3.5862
  'ein ': 3.4514

Top 5 subwords contributing to Bokmål classification:
  'ikke ': -3.2680
  'sier ': -3.1994
  'jeg ': -3.0549
  'fra ': -2.8995
  'Jeg ': -2.7088
